In [3]:
#@title Setup
!pip install torch datasets

import os
import random
import uuid

from datasets import load_dataset, Dataset, DatasetDict

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/a2/73/a68704750a7679d0b6d3ad7aa8d4da8e14e151ae82e6fee774e6e0d05ec8/urllib3-2.2.1-py3-none-any.whl.metadata
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
botocore 1.29.76 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.2.1 which is incompatible.
kaggle 1.5.8 requires urllib3<1.25,>=1.21.1, but you have urllib3 2.2.1 which is incompatible.
transformers 4.32.1 requires tokenizers!=0.11.3,<0.14,>=0.11.1, but you have tokenizers 0.15.2 which is incompatible.


ImportError: cannot import name 'DEFAULT_CIPHERS' from 'urllib3.util.ssl_' (C:\Users\Huayu Yang\AppData\Roaming\Python\Python311\site-packages\urllib3\util\ssl_.py)

In [ ]:
#@title Variables

CNN_PROMPT_PREFIX = "Summarize the following CNN article:\n"
SENTIMENT_PROMPT_PREFIX = "What's the sentiment of the sentence:\n"
NER_PROMPT_PREFIX = "Recognize the named entities from the sentence:\n"

CATEGORY_SUMMARIZTION = "Summarization"
CATEGORY_QUESTION_ANSWER = "Question&Answer"
CATEGORY_SENTIMENT_ANALYSIS = "SentimentAnalysis"
CATEGORY_NAMED_ENTITY = "NamedEntity"

COLUMN_PROMPT = "Prompt"
COLUMN_TARGET = "Target"
COLUMN_ID = "PromptId"
COLUMN_CLASSIFIER = "Classifier"


train_data_list = []
val_data_list = []
test_data_list = []

In [ ]:
#@title CNN daily mail
# https://huggingface.co/datasets/cnn_dailymail

cnn_dataset = load_dataset("cnn_dailymail", '3.0.0')

print(cnn_dataset)

def append_cnn_data_list(result_list, split):
  for row in cnn_dataset[split]:
    new_row = {}
    new_row[COLUMN_CLASSIFIER] = CATEGORY_SUMMARIZTION
    new_row[COLUMN_PROMPT] = CNN_PROMPT_PREFIX + row['article']
    new_row[COLUMN_TARGET] = row['highlights']
    new_row[COLUMN_ID] = CATEGORY_SUMMARIZTION + "/" + row['id']

    result_list.append(new_row)

append_cnn_data_list(train_data_list, 'train')
append_cnn_data_list(val_data_list, 'validation')
append_cnn_data_list(test_data_list, 'test')

In [ ]:
#@title Question Answer
# https://huggingface.co/datasets/rajpurkar/squad

squad_dataset = load_dataset("rajpurkar/squad")
print(squad_dataset)

def append_squad_data_list(result_list, split):
  for row in squad_dataset[split]:
    new_row = {}
    new_row[COLUMN_CLASSIFIER] = CATEGORY_QUESTION_ANSWER
    new_row[COLUMN_PROMPT] = row['title'] + "\n" + row['context'] + "\n" + row['question']
    new_row[COLUMN_TARGET] = ','.join(row['answers']['text'])
    new_row[COLUMN_ID] = CATEGORY_QUESTION_ANSWER + "/" + row['id']

    result_list.append(new_row)

append_squad_data_list(train_data_list, 'train')
append_squad_data_list(val_data_list, 'validation')

In [ ]:
#@title Sentiment Analysis
# https://huggingface.co/datasets/financial_phrasebank

financial_dataset = load_dataset("financial_phrasebank", "sentences_50agree")
print(financial_dataset)

for row in financial_dataset['train']:
  new_row = {}
  new_row[COLUMN_CLASSIFIER] = CATEGORY_SENTIMENT_ANALYSIS
  new_row[COLUMN_PROMPT] = SENTIMENT_PROMPT_PREFIX + row['sentence']
  if row['label'] == 0:
    new_row[COLUMN_TARGET] = 'negative'
  elif row['label'] == 1:
    new_row[COLUMN_TARGET] = 'neutral'
  else:
    new_row[COLUMN_TARGET] = 'positive'

  new_row[COLUMN_ID] = CATEGORY_SENTIMENT_ANALYSIS + "/" + str(uuid.uuid4())

  rand = random.randint(1, 10)
  if rand == 1:
    val_data_list.append(new_row)
  elif rand == 2:
    test_data_list.append(new_row)
  else:
    train_data_list.append(new_row)


In [ ]:
#@title Named Entity Recognition
# https://huggingface.co/datasets/conll2003

ner_dataset = load_dataset("conll2003")
print(ner_dataset)

NER_DICT = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
NER_DICT = {v: k for k, v in NER_DICT.items()}

def append_ner_data_list(result_list, split):
  for row in ner_dataset[split]:
    new_row = {}
    new_row[COLUMN_CLASSIFIER] = CATEGORY_NAMED_ENTITY
    new_row[COLUMN_PROMPT] = NER_PROMPT_PREFIX + ' '.join(row['tokens'])
    answers = []
    for idx, ner_tag in enumerate(row['ner_tags']):
      if ner_tag == 0:
        continue

      answers.append(row['tokens'][idx] + ":" + NER_DICT[ner_tag])

    new_row[COLUMN_TARGET] = ','.join(answers)
    new_row[COLUMN_ID] = CATEGORY_NAMED_ENTITY + "/" + row['id']

    result_list.append(new_row)

append_ner_data_list(train_data_list, 'train')
append_ner_data_list(val_data_list, 'validation')
append_ner_data_list(test_data_list, 'test')

In [ ]:
#@title Create Dataset
# huggingface-cli login
# print(train_data_list[:1])
# for data in train_data_list:
#   for k,v in data.items():
#     if (not isinstance(k, str)):
#       print(type(k))
#       print(k)
#     if (not isinstance(v, str)):
#       print(type(v))
#       print(v)


train_ds = Dataset.from_list(train_data_list)
val_ds = Dataset.from_list(val_data_list)
test_ds = Dataset.from_list(test_data_list)

dd = DatasetDict({"train": train_ds, "validation": val_ds, "test_ds": test_ds})


# dd.push_to_hub("ccss4/hw5-finetuning")